### Setups
- Task 0 for Task-A, 1 for Task-B, 2 for Task-C

In [1]:
MAX_LEN = 80
TASK = 1

### Datapaths

In [2]:
datadir = "../data/"
trainpath = datadir+"olid-training-v1.0.tsv"
testpath = datadir+"testset-level{}.tsv".format("abc"[TASK])
labelpath = datadir+"labels-level{}.csv".format("abc"[TASK])

### Prerequisites

In [3]:
# !pip install --user wordninja
# import nltk
# nltk.download('averaged_perceptron_tagger')

In [4]:
from dataset import make_data_generator
from models import SimpleLSTM, SelfAttnLSTM, Criterion
import torch
import matplotlib.pyplot as plt
plt.show()

### Make Generator
- creates generator for batches of training/validation data

In [5]:
(training_set, training_gen),(validation_set, validation_gen)\
= make_data_generator(trainpath, task=TASK, val_split=0.1, batch_size=32, n_workers=4, max_len=MAX_LEN, shuffle=True)

[info] 4400 data.



[info] vocab size = 9536
[info] 3961 train. 439 valid.


In [6]:
VOCAB = (training_set.vocab, training_set.pos_vocab)
VOC_SIZE = len(VOCAB[0])
POS_SIZE = len(VOCAB[1])

### Model, Loss, Optimizer
- create a model of SimpleLSTM or SelfAttnLSTM
- create a criterion(focal loss & f1 calculations)
- create an optimizer

In [7]:
# model = SimpleLSTM(demb=100, demb_pos=128, dmodel=120, dff=150, \
#             voc_sz=VOC_SIZE, pos_voc_sz=POS_SIZE, max_len=MAX_LEN,\
#             out_class=(3 if TASK==2 else 2), dropout=0.5)
model = SelfAttnLSTM(demb=100, demb_pos=100, dmodel=120, dff=150, \
            voc_sz=VOC_SIZE, pos_voc_sz=POS_SIZE, max_len=MAX_LEN,\
            out_class=(3 if TASK==2 else 2), dropout=0.5)

model.train()
model.cuda()

criterion = Criterion(TASK, cuda=True)
optim = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9, weight_decay=1e-4)

### Validation
- call this function to run validation
- returns loss, accuracy, f1

In [8]:
def validation(model, val_gen, criterion):
    was_training = model.training
    model.eval()
    with torch.no_grad():
        first = True
        all_logits = None
        all_y = None
        
        for j, (x, pos, y) in enumerate(val_gen):
            x = x.cuda()
            pos = pos.cuda()
            y = y.cuda()

            logits = model.forward(x, pos)
            
            if first:
                all_logits = logits
                all_y = y
                first = False
            else:
                all_logits = torch.cat((all_logits, logits), dim=0)
                all_y = torch.cat((all_y, y), dim=0)
            
        loss = criterion.loss_compute(all_logits, all_y)
        accu = criterion.accu_compute(all_logits, all_y)
        f1 = criterion.f1_compute(all_logits, all_y)
    if was_training:
        model.train()
    return loss.item(), accu, f1

### Training 
- this part trains the model for specified epochs

In [9]:
EPOCHS = 10# 10 for task A, 7 for task B
print("Epoch Loss Accu ValLoss ValAccu ValF1")
for i,e in enumerate(range(EPOCHS)):
    running_loss = 0
    running_accu = 0
    running_f1 = 0
    running_step = 0
    
    for j, (x, pos, y) in enumerate(training_gen):
        x = x.cuda()
        pos = pos.cuda()
        y = y.cuda()

        logits = model.forward(x, pos)
        loss = criterion.loss_compute(logits, y)
        accu = criterion.accu_compute(logits, y)

        optim.zero_grad()
        loss.backward()
        optim.step()        
        
        running_loss += loss.item()
        running_accu += accu
        running_step += 1        
        print(loss.item(), end='\r')
        
    vloss, vaccu, vf1 = validation(model, validation_gen, criterion)
    print("{} {:.5f} {:.5f} {:.5f} {:.5f} {:.5f}".format(i, running_loss/running_step, \
                                                   running_accu/running_step, vloss, vaccu, vf1))
# torch.save(model, "tmp.pkt")

Epoch Loss Accu ValLoss ValAccu ValF1
0 0.22657 0.87903 0.23415 0.88155 0.46852


/home/george/.conda/envs/pytorch/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1 0.21177 0.88098 0.22690 0.88155 0.46852
2 0.19988 0.87911 0.23617 0.88155 0.46852
3 0.18884 0.88422 0.23687 0.86105 0.52027
4 0.18065 0.88203 0.22748 0.87472 0.51552
5 0.17230 0.89005 0.25547 0.83371 0.52456
6 0.16199 0.89272 0.25081 0.81777 0.49482
7 0.13860 0.91007 0.28574 0.84510 0.54265
8 0.12863 0.91465 0.31173 0.83599 0.50489
9 0.11404 0.92040 0.27879 0.81777 0.51433


### Testing
- this part tests the model for its performance on the test set

In [10]:
# model = torch.load("tmp.pkt")
testing_set, testing_gen = make_data_generator(testpath, task=TASK, testlabel=labelpath, \
                                               batch_size=1024, n_workers=4, load_vocabs=VOCAB,\
                                               max_len=MAX_LEN, shuffle=False)

[info] 240 data.



[info] vocab size = 9536


In [11]:
vloss, vaccu, vf1 = validation(model, testing_gen, criterion)
print("TestLoss TestAccu TestF1")
print("{:.5f} {:.5f} {:.5f}".format(vloss, vaccu, vf1))

TestLoss TestAccu TestF1
0.18956 0.86250 0.68115
